#### 문제정의
- 날씨데이터와 로또데이터를 이용해서 해당 번호가 몇등인지를 예측
- 번호공이 나오는 확률이 날씨에 영향을 받지 않을까라는 생각
- 날씨 : 기상청
- 로또 : 해당 사이트

In [4]:
import pandas as pd

lotto = pd.read_csv("./data/lotto_869.csv", encoding="euc-kr")
lotto = lotto.set_index("추첨일")
lotto.head()

ERROR! Session/line number was not unique in database. History logging moved to new session 360


,회차,1등수,1등금액,2등수,2등금액,3등수,3등금액,4등수,4등금액,5등수,5등금액,No_1,No_2,No_3,No_4,No_5,No_6,No_Bonus
추첨일,,,,,,,,,,,,,,,,,,
2019-07-27,869,10,1922582588,56,57219720,2210,1449912,108562,50000,1814332,5000,2,6,20,27,37,39,4
2019-07-20,868,6,3233804250,61,53013185,2152,1502698,109133,50000,1789491,5000,12,17,28,41,43,44,25
2019-07-13,867,5,3933755250,57,57511042,2175,1507186,109516,50000,1820552,5000,14,17,19,22,24,40,41
2019-07-06,866,9,2240409000,68,49420787,2035,1651408,104919,50000,1770017,5000,9,15,29,34,37,39,12
2019-06-29,865,13,1551729145,52,64655382,2076,1619499,104626,50000,1749234,5000,3,15,22,32,33,45,2


In [5]:
#필요한 인덱스만 잘라내기(=슬라이싱)
lotto = lotto.loc[:,["No_1", "No_2", "No_3", "No_4", "No_5", "No_6", "No_Bonus"]]
lotto.head()

,No_1,No_2,No_3,No_4,No_5,No_6,No_Bonus
추첨일,,,,,,,
2019-07-27,2,6,20,27,37,39,4
2019-07-20,12,17,28,41,43,44,25
2019-07-13,14,17,19,22,24,40,41
2019-07-06,9,15,29,34,37,39,12
2019-06-29,3,15,22,32,33,45,2


In [6]:
lotto = lotto.loc[:, "No_1":]
lotto.head()

,No_1,No_2,No_3,No_4,No_5,No_6,No_Bonus
추첨일,,,,,,,
2019-07-27,2,6,20,27,37,39,4
2019-07-20,12,17,28,41,43,44,25
2019-07-13,14,17,19,22,24,40,41
2019-07-06,9,15,29,34,37,39,12
2019-06-29,3,15,22,32,33,45,2


#### 날씨 데이터 전처리

In [7]:
weather = pd.read_csv("./data/weater_seoul_200212_20190727.csv", encoding="euc-kr")
weather = weather.set_index("일시")
weather.head()

,지점,평균기온,최저기온,최고기온,평균 풍속,평균 상대습도,평균 현지기압,합계 일조 시간,평균 전운량,평균 지면온도
일시,,,,,,,,,,
2002-12-01,108,4.8,-0.7,9.9,1.2,60.1,1012.5,4.6,2.9,1.7
2002-12-02,108,6.9,3.3,12.1,1.3,63.6,1015.7,3.7,2.0,3.1
2002-12-03,108,7.7,2.8,11.0,2.0,52.4,1011.6,0.3,7.1,2.8
2002-12-04,108,11.3,8.0,15.9,1.8,50.4,1005.5,5.2,4.9,7.0
2002-12-05,108,7.3,4.2,10.5,2.7,67.9,1010.4,0.0,8.1,6.2


In [8]:
weather = weather.loc[:,["평균기온","평균 상대습도","평균 현지기압"]]
weather.head()

,평균기온,평균 상대습도,평균 현지기압
일시,,,
2002-12-01,4.8,60.1,1012.5
2002-12-02,6.9,63.6,1015.7
2002-12-03,7.7,52.4,1011.6
2002-12-04,11.3,50.4,1005.5
2002-12-05,7.3,67.9,1010.4


#### 로또와 날씨데이터를 concat으로 병합해보자

In [9]:
#axis=1 : 가로방향으로 병합
#join="inner" : 병합하면서 결측치가 생기는 해당 데이터는 삭제
lotto_weather = pd.concat([lotto,weather], axis=1, join="inner")
lotto_weather.index.names = ['일시']
lotto_weather.head()

,No_1,No_2,No_3,No_4,No_5,No_6,No_Bonus,평균기온,평균 상대습도,평균 현지기압
일시,,,,,,,,,,
2019-07-27,2,6,20,27,37,39,4,28.4,76.6,995.7
2019-07-20,12,17,28,41,43,44,25,28.0,66.9,990.8
2019-07-13,14,17,19,22,24,40,41,24.9,67.3,992.0
2019-07-06,9,15,29,34,37,39,12,29.7,40.9,994.1
2019-06-29,3,15,22,32,33,45,2,23.1,72.0,988.3


#### 데이터에 1등 라벨 추가

In [10]:
lotto_weather["등수"] =1
lotto_weather.head()

,No_1,No_2,No_3,No_4,No_5,No_6,No_Bonus,평균기온,평균 상대습도,평균 현지기압,등수
일시,,,,,,,,,,,
2019-07-27,2,6,20,27,37,39,4,28.4,76.6,995.7,1
2019-07-20,12,17,28,41,43,44,25,28.0,66.9,990.8,1
2019-07-13,14,17,19,22,24,40,41,24.9,67.3,992.0,1
2019-07-06,9,15,29,34,37,39,12,29.7,40.9,994.1,1
2019-06-29,3,15,22,32,33,45,2,23.1,72.0,988.3,1


#### 2등 번호를 추가하고 2등
- 2등 : 당첨번호에서 각각의 번호를 보너스 번호롤 대처핸것

In [11]:
lotto_weather3 = lotto_weather.copy()

# 전체 회차만큼 반복
for i in range(len(lotto_weather)) : 
    # 회차마다 6번씩 반복해서 각 번호를 보너스 번호로 변경
    for j in range(6) :
        # 한 회차 데이터만 가져온다
        temp = lotto_weather.iloc[[i], :]
        
        # 한 회차 데이터에서 j 번째 번호를 보너스 번호로 변경
        temp.iloc[:, j] = temp.loc[:, "No_Bonus"]
        
        # 등수를 추가
        temp.loc[:, "등수"] = 2
        
        # 변경된 데이터는 원본 데이터에 추가 (세로방향)
        lotto_weather3 = pd.concat([lotto_weather3, temp])

C:\Users\SM2139\anaconda3\envs\deep01\lib\site-packages\pandas\core\indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
C:\Users\SM2139\anaconda3\envs\deep01\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [12]:
lotto_weather3.shape

(6083, 11)

In [13]:
lotto_weather3.tail()

,No_1,No_2,No_3,No_4,No_5,No_6,No_Bonus,평균기온,평균 상대습도,평균 현지기압,등수
일시,,,,,,,,,,,
2002-12-07,10,16,29,33,37,40,16,5.2,63.9,1014.5,2
2002-12-07,10,23,16,33,37,40,16,5.2,63.9,1014.5,2
2002-12-07,10,23,29,16,37,40,16,5.2,63.9,1014.5,2
2002-12-07,10,23,29,33,16,40,16,5.2,63.9,1014.5,2
2002-12-07,10,23,29,33,37,16,16,5.2,63.9,1014.5,2


#### 3등 추가
- 1등에서 5개 맞는 번호
- 회차마다 3등 갯수 : 869 *6 * 39

#### 4등 추가
- 1등 데이터에서 4개를 맞춘 것

#### 5등 추가
- 1등 데이터에서 3개를 맞춘 것

#### No_Bonus컬럼 삭제

In [14]:
del lotto_weather3["No_Bonus"]
lotto_weather3

,No_1,No_2,No_3,No_4,No_5,No_6,평균기온,평균 상대습도,평균 현지기압,등수
일시,,,,,,,,,,
2019-07-27,2,6,20,27,37,39,28.4,76.6,995.7,1
2019-07-20,12,17,28,41,43,44,28.0,66.9,990.8,1
2019-07-13,14,17,19,22,24,40,24.9,67.3,992.0,1
2019-07-06,9,15,29,34,37,39,29.7,40.9,994.1,1
2019-06-29,3,15,22,32,33,45,23.1,72.0,988.3,1
...,...,...,...,...,...,...,...,...,...,...
2002-12-07,10,16,29,33,37,40,5.2,63.9,1014.5,2
2002-12-07,10,23,16,33,37,40,5.2,63.9,1014.5,2
2002-12-07,10,23,29,16,37,40,5.2,63.9,1014.5,2


#### 특성 데이터와 라벨 데이터로 분리

In [15]:
X=lotto_weather3.iloc[:, :-1]
y=lotto_weather3.iloc[:, -1]

In [16]:
y.unique()

array([1, 2], dtype=int64)

#### 다진분류 : 원핫 인코딩

In [17]:
y_en = pd.get_dummies(y)
y_en.shape

(6083, 2)

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()

# 입력층
model.add(Dense(units=32, input_dim=9, activation="relu"))

# 은닉층
model.add(Dense(units=64, activation="relu"))
model.add(Dense(units=32, activation="relu"))
#model.add(Dropout(0.5))

# 퍼셉트론 중에서 50%는 사용하지 않겠다 
# epoch마다 사용하지 않는 퍼셉트론을 다를 수 있다


# 출력층
model.add(Dense(units=2, activation="softmax"))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                320       
_________________________________________________________________
dense_5 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 66        
Total params: 4,578
Trainable params: 4,578
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(loss="categorical_crossentropy",
               optimizer="adam",
               metrics=["accuracy"])

In [20]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [21]:
h = model.fit(X, y_en, epochs=100, batch_size=100,
             validation_split=0.3)

Train on 4258 samples, validate on 1825 samples
Epoch 1/100
4258/4258 [==============================] - 1s 159us/sample - loss: 2.1975 - accuracy: 0.6858 - val_loss: 0.1106 - val_accuracy: 0.9748
Epoch 2/100
4258/4258 [==============================] - 0s 21us/sample - loss: 0.7782 - accuracy: 0.7031 - val_loss: 0.0765 - val_accuracy: 0.9962
Epoch 3/100
4258/4258 [==============================] - 0s 20us/sample - loss: 0.7573 - accuracy: 0.7076 - val_loss: 0.1016 - val_accuracy: 0.9973
Epoch 4/100
4258/4258 [==============================] - 0s 20us/sample - loss: 0.5245 - accuracy: 0.7605 - val_loss: 0.0427 - val_accuracy: 1.0000
Epoch 5/100
4258/4258 [==============================] - 0s 21us/sample - loss: 0.7601 - accuracy: 0.7107 - val_loss: 0.2630 - val_accuracy: 0.9605
Epoch 6/100
4258/4258 [==============================] - 0s 21us/sample - loss: 0.5240 - accuracy: 0.7586 - val_loss: 0.0928 - val_accuracy: 1.0000
Epoch 7/100
4258/4258 [==============================] - 0s 21u

Epoch 56/100
4258/4258 [==============================] - 0s 20us/sample - loss: 0.5221 - accuracy: 0.7468 - val_loss: 0.3121 - val_accuracy: 0.8718
Epoch 57/100
4258/4258 [==============================] - 0s 20us/sample - loss: 0.4741 - accuracy: 0.7715 - val_loss: 0.0795 - val_accuracy: 1.0000
Epoch 58/100
4258/4258 [==============================] - 0s 20us/sample - loss: 0.4786 - accuracy: 0.7588 - val_loss: 0.1663 - val_accuracy: 0.9879
Epoch 59/100
4258/4258 [==============================] - 0s 22us/sample - loss: 0.4821 - accuracy: 0.7616 - val_loss: 0.0930 - val_accuracy: 1.0000
Epoch 60/100
4258/4258 [==============================] - 0s 19us/sample - loss: 0.4278 - accuracy: 0.7811 - val_loss: 0.1769 - val_accuracy: 0.9874
Epoch 61/100
4258/4258 [==============================] - 0s 21us/sample - loss: 0.4443 - accuracy: 0.7783 - val_loss: 0.1371 - val_accuracy: 0.9984
Epoch 62/100
4258/4258 [==============================] - 0s 20us/sample - loss: 0.5263 - accuracy: 0.7464

In [ ]:
acc= h.history["accuracy"]
val_acc=h.history["val_accuracy"]
e = range(1, len(acc)+1)
plt.plot(e, acc, "-", color="red", label="train_acc")
plt.plot(e, acc, "-", color="blue", label="test_acc")
plt.legend()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\SM2139\anaconda3\envs\deep01\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-22-cf2928d3e2b3>", line 4, in <module>
    plt.plot(e, acc, "-", color="red", label="train_acc")
NameError: name 'plt' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\SM2139\anaconda3\envs\deep01\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\SM2139\anaconda3\envs\deep01\lib\site-packages\IPython\core\ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_

In [ ]:
lottoData = np.array([1, 2, 3, 4, 5, 6, 35., 80., 990.]).reshape(1,9)

pred = model.predict(lottoData)

pred

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\SM2139\anaconda3\envs\deep01\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-22-6096001ad43e>", line 1, in <module>
    lottoData = np.array([1, 2, 3, 4, 5, 6, 35., 80., 990.]).reshape(1,9)
NameError: name 'np' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\SM2139\anaconda3\envs\deep01\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\SM2139\anaconda3\envs\deep01\lib\site-packages\IPython\core\ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\SM2139\anaconda3\envs\deep01\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-22-6096001ad43e>", line 1, in <module>
    lottoData = np.array([1, 2, 3, 4, 5, 6, 35., 80., 990.]).reshape(1,9)
NameError: name 'np' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\SM2139\anaconda3\envs\deep01\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\SM2139\anaconda3\envs\deep01\lib\site-packages\IPython\core\interactiveshell.py", line 3263, in run_ast_nodes
    if (await self.run_code(code, result,  